In [1]:
import numpy as np
import sys
sys.path.append('../../pysimARG')
from clonal_genealogy import ClonalTree
from ClonalOrigin_nodes import ClonalOrigin_nodes
import ClonalOrigin_pair
from tree import tree

In [2]:
tree1 = ClonalTree(n=5)

In [3]:
print(tree1)

ClonalTree(n=5)


In [4]:
n = tree1.n
clonal_edge = tree1.edge.copy()
clonal_node_height = tree1.node_height.copy()

In [5]:
rho_site = 1.0
L = 100
delta = 5.0
k = 20

In [6]:
clonal_edge

array([[6.        , 2.        , 0.06717392],
       [6.        , 3.        , 0.06717392],
       [7.        , 6.        , 0.13015864],
       [7.        , 5.        , 0.06298472],
       [8.        , 7.        , 0.59042363],
       [8.        , 4.        , 0.59042363],
       [9.        , 8.        , 0.6369118 ],
       [9.        , 1.        , 0.6369118 ]])

In [7]:
np.unique(clonal_edge[:, 0])

array([6., 7., 8., 9.])

In [8]:
np.unique(clonal_edge[:, 1])

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [17]:
clonal_node_height[0:8]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.06717392, 0.13015864, 0.59042363])

In [ ]:
# Tree length
tree_length = np.sum(clonal_edge[:, 2])

# Initialize recombination edges
nrow_max = 1000
# Columns: b_edge, b_height, a_edge, a_height, x, y
recomb_edge = np.full((nrow_max, 6), np.nan)

# Add recombination sequentially
n_recomb = 0
remain_index = np.array([], dtype=int)

for i in range(1, 3):  # i = 1, 2
    if i == 1:
        R_new = np.random.poisson(rho_site * delta * tree_length / 2)
        R_old = 0
    else:  # i == 2
        survive_index = np.where(recomb_edge[:n_recomb, 5] == 1)[0] if n_recomb > 0 else np.array([], dtype=int)
        delta2 = np.sum((1 - 1/delta) ** np.arange(k))
        R_new = np.random.poisson(rho_site * delta2 * tree_length / 2)
        
        if len(survive_index) >= 0:
            R_old = np.random.binomial(len(survive_index), (1 - 1/delta) ** k) if len(survive_index) > 0 else 0
            if len(survive_index) == 1:
                remain_index = survive_index.copy()
            elif len(survive_index) > 1 and R_old > 0:
                remain_index = np.random.choice(survive_index, R_old, replace=False)
            else:
                remain_index = np.array([], dtype=int)
        else:
            R_old = 0
    
    if R_new > 0:
        # Expand matrix if needed
        if n_recomb + R_new >= nrow_max:
            recomb_edge = np.vstack([recomb_edge, np.full((nrow_max, 6), np.nan)])
            nrow_max = 2 * nrow_max
        
        # Set x and y columns
        recomb_edge[n_recomb:n_recomb + R_new, 4] = i  # x
        recomb_edge[n_recomb:n_recomb + R_new, 5] = i  # y
        
        a_rexp = np.random.exponential(1.0, size=R_new)
        
        # Simulate b_edge (similar to mutation)
        # Sample edges with probability proportional to edge length
        edge_probs = clonal_edge[:, 2] / np.sum(clonal_edge[:, 2])
        recomb_edge[n_recomb:n_recomb + R_new, 0] = np.random.choice(
            range(1, (2*n-1)), R_new, replace=True, p=edge_probs
        )
        
        for j in range(R_new):
            idx = n_recomb + j
            b_edge_idx = int(recomb_edge[idx, 0])
            
            # Simulate b_height
            recomb_edge[idx, 1] = (
                np.random.uniform(0, clonal_edge[b_edge_idx, 2]) +
                clonal_node_height[int(clonal_edge[b_edge_idx, 1])-1]
            )
            
            # Identify a_height
            # t_above_b: heights of internal nodes minus b_height
            t_above_b = clonal_node_height[n:2*n-1] - recomb_edge[idx, 1]
            
            # Get positive values (nodes above b)
            positive_mask = t_above_b >= 0
            positive_t = t_above_b[positive_mask]
            
            # i_above_b with 0 prepended
            i_above_b_full = np.concatenate([[0], np.sort(positive_t)])
            i_above_b = np.diff(i_above_b_full)
            
            # Calculate cumulative values
            num_intervals = len(i_above_b)
            lineage_counts = np.arange(num_intervals + 1, 1, -1)
            cuml_above_b = np.cumsum(i_above_b * lineage_counts)
            
            # Determine number of lineages at coalescence time
            num_lineage = (num_intervals + 1) - np.sum(a_rexp[j] > cuml_above_b)
            
            if num_lineage == (num_intervals + 1):
                recomb_edge[idx, 3] = a_rexp[j] / num_lineage + recomb_edge[idx, 1]
            else:
                idx_cuml = num_intervals - num_lineage
                recomb_edge[idx, 3] = (
                    (a_rexp[j] - cuml_above_b[idx_cuml]) / num_lineage +
                    np.sum(i_above_b[:idx_cuml + 1]) +
                    recomb_edge[idx, 1]
                )
            
            # Simulate a_edge
            if num_lineage > 1:
                a_height = recomb_edge[idx, 3]
                # Find edges that span the a_height
                pool_edge = np.where(
                    (clonal_node_height[clonal_edge[:, 0].astype(int)-1] >= a_height) &
                    (clonal_node_height[clonal_edge[:, 1].astype(int)-1] < a_height)
                )[0] + 1
                recomb_edge[idx, 2] = np.random.choice(pool_edge)
            else:
                # Root edge (using edge index 2*n-2 for 0-indexed)
                recomb_edge[idx, 2] = 2 * n - 2
    
    if R_old > 0 and len(remain_index) > 0:
        recomb_edge[remain_index, 5] = i
    
    n_recomb = n_recomb + R_new

In [54]:
recomb_edge[range(n_recomb), 2]

array([15., 16., 13., 16., 17., 17., 18., 16., 17.,  8., 17., 16., 16.,
       17., 17., 15., 15., 18., 17., 14., 14., 17., 16., 12., 17., 12.,
       16., 13., 16., 16.,  9., 18., 18., 17., 16., 17., 18.])

In [58]:
clonal_edge[:, 0]

array([10., 10., 11., 11., 12., 12., 13., 13., 14., 14., 15., 15., 16.,
       16., 17., 17., 18., 18.])

In [59]:
# Handle case with no recombination
if n_recomb == 0:
    edge = clonal_edge
    edge_mat = np.full((2 * (n - 1), 2), True)
    node_height = clonal_node_height
    node_mat = np.full((2 * n - 1, 2), True)
    node_clonal = np.full(2 * n - 1, True)
    sum_time = np.max(clonal_node_height)

# Trim recomb_edge to actual size
recomb_edge = recomb_edge[:n_recomb, :]

# Build full ARG
node_max = 2 * n - 1 + 3 * n_recomb
edge_max = 2 * (n - 1) + 4 * n_recomb

edge_matrix = np.full((edge_max, 3), np.nan)
edge_mat_index = np.full(edge_max, np.nan)
node_mat = np.full((node_max, 2), np.nan)
node_info = np.full((node_max, 4), np.nan)
# Columns: index, height, recomb, clonal

node_mat[:n, :] = True
node_info[:, 0] = np.arange(node_max)

# Set clonal node info
node_info[:2*n-1, 1] = clonal_node_height
node_info[:2*n-1, 2] = 0
node_info[:2*n-1, 3] = 1  # True -> 1

# Set recombination out nodes (b nodes)
# Interleave: for each recomb event, add two nodes at b_height
for r in range(n_recomb):
    base_idx = 2 * n - 1 + 2 * r
    node_info[base_idx, 1] = recomb_edge[r, 1]      # b_height
    node_info[base_idx, 2] = -(r + 1)               # negative recomb index (1-indexed)
    node_info[base_idx, 3] = 1                      # clonal = True
    
    node_info[base_idx + 1, 1] = recomb_edge[r, 1]  # same b_height
    node_info[base_idx + 1, 2] = np.nan             # NA
    node_info[base_idx + 1, 3] = 0                  # clonal = False

# Set recombination in nodes (a nodes)
for r in range(n_recomb):
    idx = 2 * n - 1 + 2 * n_recomb + r
    node_info[idx, 1] = recomb_edge[r, 3]           # a_height
    node_info[idx, 2] = r + 1                       # positive recomb index (1-indexed)
    node_info[idx, 3] = 1                           # clonal = True

# Sort by height
sort_order = np.argsort(node_info[:, 1])
node_info = node_info[sort_order, :]

# Recombination nodes on every edge
# Use 1-indexed edge indices to match R behavior
recomb_node = []
for edge_idx in range(2 * n - 1):
    # Convert to 1-indexed for ClonalOrigin_nodes
    nodes = ClonalOrigin_nodes(recomb_edge, edge_idx)
    recomb_node.append(nodes)

In [63]:
node_info

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 2.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 3.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 4.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 5.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 6.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 7.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 8.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 9.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 6.60000000e+01,  2.12384613e-03,             nan,
         0.00000000e+00],
       [ 6.50000000e+01,  2.12384613e-03, -2.40000000e+01,
      

In [65]:
node_info[:, 2]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  nan,
       -24., -10.,  nan, -29.,  nan,  nan, -31.,   0.,  -1.,  nan,   0.,
         0., -21.,  nan,   0., -28.,  nan, -12.,  nan,  -3.,  nan,  29.,
        12.,  10.,  28.,  nan, -26.,  24.,  31.,  26.,   0.,   0., -16.,
        nan,   3.,   0.,  -6.,  nan,  -4.,  nan,   4.,   1.,  nan, -17.,
        16., -20.,  nan,  21.,  20.,  -7.,  nan,  -2.,  nan,  17., -37.,
        nan,   0., -34.,  nan,  nan, -25.,  25., -11.,  nan,  nan, -15.,
         6., -13.,  nan,  nan, -23.,  nan, -33.,  nan, -36.,  15.,  23.,
        -5.,  nan, -19.,  nan,  13.,  11., -22.,  nan,  nan, -35., -30.,
        nan,  30.,   5.,  36.,  nan, -27.,  27.,  19.,  nan, -14.,  35.,
        22.,  -9.,  nan,   2.,  -8.,  nan,   8.,  34.,  14.,   9., -32.,
        nan, -18.,  nan,   0.,   7.,  37.,  32.,  18.,  33.])

In [66]:
np.sum(node_info[:, 2]==0)

np.int64(19)

In [75]:
recomb_edge[:, 2]

array([15., 16., 13., 16., 17., 17., 18., 16., 17.,  8., 17., 16., 16.,
       17., 17., 15., 15., 18., 17., 14., 14., 17., 16., 12., 17., 12.,
       16., 13., 16., 16.,  9., 18., 18., 17., 16., 17., 18.])

In [ ]:
# Build ARG edges and track ancestral material
i = n  # Start after leaf nodes (0-indexed)
edge_index = 0

while i < node_max:
    recomb_val = node_info[i, 2]
    
    if recomb_val == 0:
        # Clonal tree node
        node_index = int(node_info[i, 0])
        leaf_edge = np.where(clonal_edge[:, 0] == node_index)[0]
        leaf_index = np.full(2, np.nan)
        leaf_node = np.full(2, np.nan)
        
        for le_idx in range(2):
            le = leaf_edge[le_idx]
            if len(recomb_node[le]) > 0:
                # Target node is last element
                tar_node = recomb_node[le][-1]
                leaf_index[le_idx] = np.where(node_info[:, 2] == tar_node)[0][0]
                leaf_node[le_idx] = node_info[int(leaf_index[le_idx]), 0]
            else:
                leaf_node[le_idx] = clonal_edge[le, 1]
                leaf_index[le_idx] = np.where(node_info[:, 0] == leaf_node[le_idx])[0][0]
        
        # Append edges
        edge_matrix[edge_index:edge_index+2, 0] = i
        edge_matrix[edge_index:edge_index+2, 1] = leaf_index
        edge_matrix[edge_index:edge_index+2, 2] = node_info[i, 1] - node_info[leaf_index.astype(int), 1]
        edge_mat_index[edge_index:edge_index+2] = leaf_index
        
        # Append root node material
        li0, li1 = int(leaf_index[0]), int(leaf_index[1])
        node_mat[i, :] = np.logical_or(
            np.nan_to_num(node_mat[li0, :], nan=0).astype(bool),
            np.nan_to_num(node_mat[li1, :], nan=0).astype(bool)
        )
        
        edge_index += 2
        i += 1
        
    elif recomb_val < 0:
        # Recombination edge out node
        node_index = node_info[i:i+2, 0].astype(int)
        recomb_idx = int(abs(node_info[i, 2])) - 1  # Convert to 0-indexed
        leaf_edge = int(recomb_edge[recomb_idx, 0])
        
        tar_node_pos = np.where(recomb_node[leaf_edge] == node_info[i, 2])[0]
        if len(tar_node_pos) > 0:
            tar_node_pos = tar_node_pos[0]
            if tar_node_pos == 0:
                leaf_node = clonal_edge[leaf_edge, 1]
            else:
                prev_node = recomb_node[leaf_edge][tar_node_pos - 1]
                leaf_node = node_info[np.where(node_info[:, 2] == prev_node)[0][0], 0]
        else:
            leaf_node = clonal_edge[leaf_edge, 1]
        
        leaf_index = int(np.where(node_info[:, 0] == leaf_node)[0][0])
        
        # Append edges
        edge_matrix[edge_index:edge_index+2, 0] = [i, i + 1]
        edge_matrix[edge_index:edge_index+2, 1] = leaf_index
        edge_matrix[edge_index:edge_index+2, 2] = node_info[i, 1] - node_info[leaf_index, 1]
        edge_mat_index[edge_index:edge_index+2] = [i, i + 1]
        
        x = int(recomb_edge[recomb_idx, 4])
        y = int(recomb_edge[recomb_idx, 5])
        
        # Append root node material
        node_mat[i:i+2, :] = False
        # x:y in R is inclusive, in Python x-1:y is equivalent for 1-indexed to 0-indexed
        node_mat[i + 1, x-1:y] = np.nan_to_num(node_mat[leaf_index, x-1:y], nan=0).astype(bool)
        
        # For the complement (-(x:y) in R means all except x:y)
        mask = np.ones(2, dtype=bool)
        mask[x-1:y] = False
        node_mat[i, mask] = np.nan_to_num(node_mat[leaf_index, mask], nan=0).astype(bool)
        
        edge_index += 2
        i += 2
        
    elif recomb_val > 0:
        # Recombination edge in node
        node_index = int(node_info[i, 0])
        recomb_idx = int(node_info[i, 2]) - 1  # Convert to 0-indexed
        leaf_edge = int(recomb_edge[recomb_idx, 2])
        
        tar_node_pos = np.where(recomb_node[leaf_edge] == node_info[i, 2])[0]
        if len(tar_node_pos) > 0:
            tar_node_pos = tar_node_pos[0]
            if tar_node_pos == 0:
                if leaf_edge == 2 * n - 2:  # Root edge (0-indexed)
                    leaf_node = 2 * n - 2
                else:
                    leaf_node = clonal_edge[leaf_edge, 1]
            else:
                prev_node = recomb_node[leaf_edge][tar_node_pos - 1]
                leaf_node = node_info[np.where(node_info[:, 2] == prev_node)[0][0], 0]
        else:
            leaf_node = clonal_edge[leaf_edge, 1]
        
        leaf_index = np.full(2, np.nan)
        leaf_index[0] = int(np.where(node_info[:, 0] == leaf_node)[0][0])
        
        # Find the corresponding out node
        neg_recomb = -node_info[i, 2]
        out_node_idx = np.where(node_info[:, 2] == neg_recomb)[0]
        if len(out_node_idx) > 0:
            leaf_index[1] = out_node_idx[0] + 1
        
        # Append edges
        edge_matrix[edge_index:edge_index+2, 0] = i
        edge_matrix[edge_index:edge_index+2, 1] = leaf_index
        edge_matrix[edge_index:edge_index+2, 2] = node_info[i, 1] - node_info[leaf_index.astype(int), 1]
        edge_mat_index[edge_index:edge_index+2] = leaf_index
        
        # Append root node material
        li0, li1 = int(leaf_index[0]), int(leaf_index[1])
        node_mat[i, :] = np.logical_or(
            np.nan_to_num(node_mat[li0, :], nan=0).astype(bool),
            np.nan_to_num(node_mat[li1, :], nan=0).astype(bool)
        )
        
        edge_index += 2
        i += 1
    else:
        # NaN case - skip
        i += 1

# Store results
edge = edge_matrix
valid_indices = ~np.isnan(edge_mat_index)
edge_mat = node_mat[edge_mat_index[valid_indices].astype(int), :]
node_height = node_info[:, 1]
node_mat = node_mat
node_clonal = node_info[:, 3].astype(bool)
sum_time = node_info[node_max - 1, 1]
               

IndexError: index 18 is out of bounds for axis 0 with size 18